In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

import matplotlib
import matplotlib.pyplot as plt
test = pd.read_csv("../input/test.csv")
train = pd.read_csv("../input/train.csv")

class data:
    '''A class to handle input data.
    
    Syntax:
        Returns tuple (X, y, subject).
        X, y, subject = dt1.s // Training set, small non-changing (no shuffle)
        X, y, subject = dt1.f // Full training set 
        X, y, subject = dt2.s // Test set, small non-changing (no shuffle)
        X, y, subject = dt2.f // Full test set
        
    '''
    s: (None, None, None) # Small data set
    f: (None, None, None) # Full data set
    fi_test: (None, None, None) # Trimmmed data set with 30 most important features
    fi_train: (None, None, None) # Trimmmed data set with 30 most important features
    #all_but_one_subject: [] # Array of data frames.
    def __init__(self, test_or_train="Train", subject=False):
        '''Initialize data class.
        
        Initializes test or train and whether to include subject column.
        '''
        global test, train
        if test_or_train == "Train":
            ds = train
        else:
            ds = test
        if subject:
            num_cols = 562
        else:
            num_cols = 561
        # Small non-changing data set (no shuffle)
        # 20 rows.
        self.s = (ds.iloc[0:100,0:num_cols],
                  ds['Activity'].iloc[0:100],
                  ds['subject'].iloc[0:100])
        # Full data
        self.f = (ds.iloc[:,0:num_cols], ds['Activity'], ds['subject'])
        # If train, then populate feat_importance
        if test_or_train == 'Train':
            self.set_fi()
        
    def set_fi(self):
        '''Creates full training set with only the most important features included.
        
        Todo: Graph feature importance to see how each feature ranks high to low.
        '''
        from sklearn.ensemble import ExtraTreesClassifier
        from sklearn.feature_selection import SelectFromModel
        from sklearn.utils import shuffle

        X, y, subject = self.f
        print('Initial data:', X.shape)
        # Outputs: (7352, 561)
        
        # Perform feature trim actions on test set.
        # Does not include subject column.
        X_test, y_test = (test.iloc[:,0:561], test['Activity'])
        
        clf = ExtraTreesClassifier()
        clf = clf.fit(X, y)
        # Outputs: array([ 0.04...,  0.05...,  0.4...,  0.4...])
        model = SelectFromModel(clf, prefit=True)
        X_new = model.transform(X)
        X_test = model.transform(X_test)
        print('New data:', X_new.shape)
        # Outputs: (7352, 101)
        s1 = model.get_support() # shows kept and removed indexes.
        
        clf = ExtraTreesClassifier()
        clf = clf.fit(X_new, y)
        model = SelectFromModel(clf, prefit=True)
        X_new = model.transform(X_new)
        X_test = model.transform(X_test)
        print('New data x new:', X_new.shape)
        print('New data x test:', X_test.shape)
        # Outputs: (7352, 35)
        s2 = model.get_support()
        
        self.fi_test = (X_test, y, subject)
        self.fi_train = (X_new, y, subject)
        
        # Match up the support columns
        # (Not using.)
        #counts1 = 0
        #counts2 = 0
        #for i in s1:
        #    if i == True: # All trues are in s2.
        #        if s2[counts2] == False:
        #            s1[counts1] = False
        #        counts2+=1
        #    counts1+=1
        


dt1 = data("Train", False)
dt2 = data("Test", False)


In [ ]:
# Loop through and get 29/30 subjects, removing one per loop.
# df[df['A'].isin([3, 6])]
# df1.loc[:, df1.loc['a'] > 0]
#x = train.loc[:, train['subject'] != 1]
x = train.iloc[[1, 3, 5], :]
print(x.shape)
x = train[train['subject'].isin([1])]
print(x.shape)
x = train[~train['subject'].isin([1])]
print(x.shape)
print(train.shape)
#n = pd.DataFrame(train.values - x.values, columns=train.columns)
#print(n.shape)
#x = train.iloc[train.Subject.isin([30, 29]), 0]
#print(x.shape)
#x = train.iloc[train.subject.isin([30]), :]
#print(x.shape)
#print('--')
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
#X, y, _ = dt1.f
target_names = ['STANDING', 'SITTING', 'LAYING', 'WALKING_UPSTAIRS', 'WALKING', 'WALKING_DOWNSTAIRS']
for i in range(1,30):
    print('Removing subject:',i)
    x = train[~train['subject'].isin([i])]
    y = x['Activity']
    x = x.iloc[:,0:561] # Remove subject column (col 562).
    # Default layers: 5, 2
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                         hidden_layer_sizes=(50, 20), random_state=1)

    clf.fit(x, y)
    
    X2, y2, _ = dt2.f
    z = clf.predict(X2)
    print(classification_report(y, z, target_names=target_names))
    
    #x = train.subject.isin([i])
    #print('Rows for subject ',i,'=',x.shape)
    #data = train[~train.subject.isin([i])]
    #print(data.shape)

In [ ]:
dt1.f[0].head(3)
# Note included 'subject' column.

In [ ]:
dt1.f[1].head(3)

In [ ]:
np.unique(dt1.f[1])

In [ ]:
dt1.s[0].head(3)

# From http://scikit-learn.org/stable/modules/feature_selection.html

# 1.13.1
VarianceThreshold: Removes all features whose variance doesn’t meet some threshold.

There are 562 features in total.

With 0.8 variance, VarianceThreshold removes the first array items in this example:

X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]

As the variance is five 0's and one 1, or 5/6, or 80%.

Note: Variance[X]=p(1-p).

According to this formula, variance tops out at 0.25 (when p=0.5).

Note: With V[X]= ... all but one feature is removed.

In [ ]:
# Training
from sklearn.feature_selection import VarianceThreshold
X, y, _ = dt1.f
p = 0
output = [[],[]]
while p <= 0.52:
    sel = VarianceThreshold(threshold=(p*(1-p)))
    v = sel.fit_transform(X)
    output[0].append('{0:.2f}'.format(p*(1-p)))
    output[1].append(len(v[0]))
    p = p+0.01
x_points0 = output[0]
y_points0 = output[1]

# Testing
X, y, _ = dt2.f
p = 0
output = [[],[]]
while p <= 0.52:
    sel = VarianceThreshold(threshold=(p*(1-p)))
    v = sel.fit_transform(X)
    output[0].append('{0:.2f}'.format(p*(1-p)))
    output[1].append(len(v[0]))
    p = p+0.01
fig = plt.figure()
ax = fig.add_subplot(111)
x_points1 = output[0]
y_points1 = output[1]
p = ax.plot(x_points0, y_points0, 'b')
p = ax.plot(x_points1, y_points1, 'r')
ax.set_xlabel('Variance')
ax.set_ylabel('Number of features')
ax.set_title('Variance Testing Set')
fig.show()

The number of features ranges from 35 (with variance=0.25) to 562 (with variance=0).

In [ ]:
# Training
fig = plt.figure()
ax = fig.add_subplot(111)
#print (dt1.s[0].items())

ax.plot(dt1.s[0], 'b')

#for row in dt1.s[0].iterrows(): #s[0]=features,s[1]=labels
        #ax.plot(c, row, 'b')
        #ax.plot(x_points1, y_points1, 'r')
        #c+=1
#ax.set_xlabel('features')
#ax.set_ylabel('Number of features')
#ax.set_title('Variance Testing Set')
fig.show()

# Plotting Learning Curves
http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

In [ ]:
# Define: plot_learning_curve()

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
# Small data set
# Variance threshold (p=0.2).
X, y, subject = dt1.s
#y = subject
#X = [subject,subject]
#print(X)
#print(y)
# Reshape your data either using array.reshape(-1, 1)
# if your data has a single feature or array.reshape(1, -1)
# if it contains a single sample.
#X.reshape(-1, 1)
#p = 0.2
#sel = VarianceThreshold(threshold=(p*(1-p)))
#X = sel.fit_transform(subject)
#print('V[X] =', '{0:.2f}'.format(p*(1-p)),
    #': num features =',len(X[0]))

title = "Training: Learning Curves (Naive Bayes) + VarianceThreshold"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
estimator = GaussianNB()
plot_learning_curve(estimator, title, X, y, ylim=(0.2, 1.01), cv=cv, n_jobs=40)

title = "Training: Learning Curves (SVM, RBF kernel, $\gamma=0.001$) + VarianceThreshold"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=1, test_size=0.02, random_state=0)
print('cv',cv)
estimator = SVC(gamma=0.001)
plot_learning_curve(estimator, title, X, y, (0.2, 1.01), cv=cv, n_jobs=40)

plt.show()

# How well does the subject (1-30) predict the activity?

In [ ]:

#..

# Neural network
# http://scikit-learn.org/stable/modules/neural_networks_supervised.html

In [ ]:
from sklearn.neural_network import MLPClassifier
X, y, _ = dt1.f
#X = [[0., 0.], [1., 1.]]
#y = [0, 1]
# Default layers: 5, 2
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X, y)

In [ ]:
X, y, _ = dt2.f
z = clf.predict(X)
from sklearn.metrics import classification_report
target_names = ['STANDING', 'SITTING', 'LAYING', 'WALKING_UPSTAIRS', 'WALKING', 'WALKING_DOWNSTAIRS']
print(classification_report(y, z, target_names=target_names))

In [ ]:
from sklearn.neural_network import MLPClassifier
import time
X, y, _ = dt1.f
#X = [[0., 0.], [1., 1.]]
#y = [0, 1]
# Default hidden layers: 5, 2
# 500, 200 takes longer but performs slightly worse than 50, 20.
t = time.time()
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(50, 20), random_state=1)

clf.fit(X, y)
print('Elapsed time: {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

In [ ]:
X, y, _ = dt2.f
z = clf.predict(X)
from sklearn.metrics import classification_report
target_names = ['STANDING', 'SITTING', 'LAYING', 'WALKING_UPSTAIRS', 'WALKING', 'WALKING_DOWNSTAIRS']
print(classification_report(y, z, target_names=target_names))

In [ ]:
# Loop through most important features and test removing one.
X, y, _ = dt1.fi_train
print(X.shape)
print(y.shape)

from sklearn.neural_network import MLPClassifier
import time
# Default hidden layers: 5, 2
# 500, 200 takes longer but performs slightly worse than 50, 20.
t = time.time()
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(50, 20), random_state=1)

clf.fit(X, y)
print('Elapsed time: {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

X, y, _ = dt1.fi_test
print(X.shape)
print(y.shape)
#X, y, _ = dt2.s
z = clf.predict(X)
from sklearn.metrics import classification_report
target_names = ['STANDING', 'SITTING', 'LAYING', 'WALKING_UPSTAIRS', 'WALKING', 'WALKING_DOWNSTAIRS']
print(classification_report(y, z, target_names=target_names))

# K Nearest Neighbor

In [ ]:
# knn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
#np.random.seed(0)
X, y, _ = dt1.f
#indices = np.random.permutation(len(X))
#iris_X_train = iris_X[indices[:-10]]
#iris_y_train = iris_y[indices[:-10]]
#iris_X_test  = iris_X[indices[-10:]]
#iris_y_test  = iris_y[indices[-10:]]

# Create and fit a nearest-neighbor classifier
knn = KNeighborsClassifier()
knn.fit(X, y) 
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
z = knn.predict(dt2.f[0])
target_names = ['STANDING', 'SITTING', 'LAYING', 'WALKING_UPSTAIRS', 'WALKING', 'WALKING_DOWNSTAIRS']
print(classification_report(dt2.f[1], z, target_names=target_names))

In [ ]:
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import matplotlib.pyplot as plt

# Full
lr = linear_model.LinearRegression()
X,y,_ = dt1.f
# get_dummies
# one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
y = pd.get_dummies(y)
# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
predicted = cross_val_predict(lr, X, y, cv=10)
# Plot
fig, ax = plt.subplots()
ax.scatter(y, predicted, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

# Small
lr = linear_model.LinearRegression()
X,y,_ = dt1.s
# get_dummies
# one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
y = pd.get_dummies(y)
# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
predicted = cross_val_predict(lr, X, y, cv=10)
# Plot
fig, ax = plt.subplots()
ax.scatter(y, predicted, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

# Full, Subject only
lr = linear_model.LinearRegression()
X,y,subject = dt1.f
X = subject.reshape(-1, 1)
# get_dummies
# one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
y = pd.get_dummies(y)
# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
predicted = cross_val_predict(lr, X, y, cv=10)
# Plot
fig, ax = plt.subplots()
ax.scatter(y, predicted, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)

ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
# Concatenating multiple feature extraction methods
# http://scikit-learn.org/stable/auto_examples/plot_feature_stacker.html#sphx-glr-auto-examples-plot-feature-stacker-py
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

X, y, _ = dt1.s

# This dataset is way too high-dimensional. Better do PCA:
pca = PCA(n_components=2)

# Maybe some original features where good, too?
selection = SelectKBest(k=1)

# Build estimator from PCA and Univariate selection:

combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])

# Use combined features to transform dataset:
X_features = combined_features.fit(X, y).transform(X)

svm = SVC(kernel="linear")

# Do grid search over k, n_components and C:

pipeline = Pipeline([("features", combined_features), ("svm", svm)])

param_grid = dict(features__pca__n_components=[1, 2, 3],
                  features__univ_select__k=[1, 2],
                  svm__C=[0.1, 1, 10])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=10)
grid_search.fit(X, y)
print(grid_search.best_estimator_)

In [ ]:
# Pipelining: chaining a PCA and a logistic regression
# http://scikit-learn.org/stable/auto_examples/plot_digits_pipe.html#sphx-glr-auto-examples-plot-digits-pipe-py
# Code source: Gaël Varoquaux
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

logistic = linear_model.LogisticRegression()

pca = decomposition.PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

digits = datasets.load_digits()
X,y,_ = dt1.f
X_digits = X
y_digits = y

# Plot the PCA spectrum
pca.fit(X_digits)

plt.figure(1, figsize=(12, 9))
plt.clf()
plt.axes([.2, .2, .7, .7])
plt.plot(pca.explained_variance_, linewidth=2)
plt.axis('tight')
plt.xlabel('n_components')
plt.ylabel('explained_variance_')

# Prediction
n_components = [20, 40, 64]
Cs = np.logspace(-4, 4, 3)

# Parameters of pipelines can be set using ‘__’ separated parameter names:
estimator = GridSearchCV(pipe,
                         dict(pca__n_components=n_components,
                              logistic__C=Cs))
estimator.fit(X_digits, y_digits)

plt.axvline(estimator.best_estimator_.named_steps['pca'].n_components,
            linestyle=':', label='n_components chosen')
plt.legend(prop=dict(size=12))
plt.show()

In [ ]:
# The Johnson-Lindenstrauss bound for embedding with random projections
# http://scikit-learn.org/stable/auto_examples/plot_johnson_lindenstrauss_bound.html#sphx-glr-auto-examples-plot-johnson-lindenstrauss-bound-py
import sys
from time import time
from sklearn.random_projection import johnson_lindenstrauss_min_dim
from sklearn.random_projection import SparseRandomProjection
from sklearn.datasets import fetch_20newsgroups_vectorized
from sklearn.datasets import load_digits
from sklearn.metrics.pairwise import euclidean_distances

# Part 1: plot the theoretical dependency between n_components_min and
# n_samples

# range of admissible distortions
eps_range = np.linspace(0.1, 0.99, 5)
colors = plt.cm.Blues(np.linspace(0.3, 1.0, len(eps_range)))

# range of number of samples (observation) to embed
n_samples_range = np.logspace(1, 9, 9)

plt.figure()
for eps, color in zip(eps_range, colors):
    min_n_components = johnson_lindenstrauss_min_dim(n_samples_range, eps=eps)
    plt.loglog(n_samples_range, min_n_components, color=color)

plt.legend(["eps = %0.1f" % eps for eps in eps_range], loc="lower right")
plt.xlabel("Number of observations to eps-embed")
plt.ylabel("Minimum number of dimensions")
plt.title("Johnson-Lindenstrauss bounds:\nn_samples vs n_components")

# range of admissible distortions
eps_range = np.linspace(0.01, 0.99, 100)

# range of number of samples (observation) to embed
n_samples_range = np.logspace(2, 6, 5)
colors = plt.cm.Blues(np.linspace(0.3, 1.0, len(n_samples_range)))

plt.figure()
for n_samples, color in zip(n_samples_range, colors):
    min_n_components = johnson_lindenstrauss_min_dim(n_samples, eps=eps_range)
    plt.semilogy(eps_range, min_n_components, color=color)

plt.legend(["n_samples = %d" % n for n in n_samples_range], loc="upper right")
plt.xlabel("Distortion eps")
plt.ylabel("Minimum number of dimensions")
plt.title("Johnson-Lindenstrauss bounds:\nn_components vs eps")

# Part 2: perform sparse random projection of some digits images which are
# quite low dimensional and dense or documents of the 20 newsgroups dataset
# which is both high dimensional and sparse

#if '--twenty-newsgroups' in sys.argv:
#    # Need an internet connection hence not enabled by default
#    data = fetch_20newsgroups_vectorized().data[:500]
#else:
#    data = load_digits().data[:500]
data,y,_ = dt1.f

n_samples, n_features = data.shape
print("Embedding %d samples with dim %d using various random projections"
      % (n_samples, n_features))

n_components_range = np.array([300, 1000, 10000])
dists = euclidean_distances(data, squared=True).ravel()

# select only non-identical samples pairs
nonzero = dists != 0
dists = dists[nonzero]

for n_components in n_components_range:
    t0 = time()
    rp = SparseRandomProjection(n_components=n_components)
    projected_data = rp.fit_transform(data)
    print("Projected %d samples from %d to %d in %0.3fs"
          % (n_samples, n_features, n_components, time() - t0))
    if hasattr(rp, 'components_'):
        n_bytes = rp.components_.data.nbytes
        n_bytes += rp.components_.indices.nbytes
        print("Random matrix with size: %0.3fMB" % (n_bytes / 1e6))

    projected_dists = euclidean_distances(
        projected_data, squared=True).ravel()[nonzero]

    plt.figure()
    plt.hexbin(dists, projected_dists, gridsize=100, cmap=plt.cm.PuBu)
    plt.xlabel("Pairwise squared distances in original space")
    plt.ylabel("Pairwise squared distances in projected space")
    plt.title("Pairwise distances distribution for n_components=%d" %
              n_components)
    cb = plt.colorbar()
    cb.set_label('Sample pairs counts')

    rates = projected_dists / dists
    print("Mean distances rate: %0.2f (%0.2f)"
          % (np.mean(rates), np.std(rates)))

    plt.figure()
    plt.hist(rates, bins=50, normed=True, range=(0., 2.), edgecolor='k')
    plt.xlabel("Squared distances rate: projected / original")
    plt.ylabel("Distribution of samples pairs")
    plt.title("Histogram of pairwise distance rates for n_components=%d" %
              n_components)

    # TODO: compute the expected value of eps and add them to the previous plot
    # as vertical lines / region

plt.show()

In [ ]:
# http://scikit-learn.org/stable/auto_examples/plot_kernel_ridge_regression.html#sphx-glr-auto-examples-plot-kernel-ridge-regression-py
# ValueError: bad input shape (80, 6)

In [ ]:
# http://scikit-learn.org/stable/auto_examples/applications/plot_model_complexity_influence.html#sphx-glr-auto-examples-applications-plot-model-complexity-influence-py
# Author: Eustache Diemert <eustache@diemert.fr>
# License: BSD 3 clause

import time
from mpl_toolkits.axes_grid1.parasite_axes import host_subplot
from mpl_toolkits.axisartist.axislines import Axes
from scipy.sparse.csr import csr_matrix

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.svm.classes import NuSVR
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from sklearn.linear_model.stochastic_gradient import SGDClassifier
from sklearn.metrics import hamming_loss

# #############################################################################
# Routines


# Initialize random generator
np.random.seed(0)


def generate_data(case, sparse=False):
    """Generate regression/classification data."""
    bunch = None
    X, y, _ = dt1.f
    #y = pd.get_dummies(y)
    #if case == 'regression':
    #    pass
    #elif case == 'classification':
    #    y = pd.get_dummies(y)
    #    pass
    
    
    #X, y = shuffle(bunch.data, bunch.target)
    offset = int(X.shape[0] * 0.8)
    X_train, y_train = X[:offset], y[:offset]
    X_test, y_test = X[offset:], y[offset:]
    if sparse:
        X_train = csr_matrix(X_train)
        X_test = csr_matrix(X_test)
    else:
        X_train = np.array(X_train)
        X_test = np.array(X_test)
    y_test = np.array(y_test)
    y_train = np.array(y_train)
    data = {'X_train': X_train, 'X_test': X_test, 'y_train': y_train,
            'y_test': y_test}
    return data


def benchmark_influence(conf):
    """
    Benchmark influence of :changing_param: on both MSE and latency.
    """
    prediction_times = []
    prediction_powers = []
    complexities = []
    for param_value in conf['changing_param_values']:
        conf['tuned_params'][conf['changing_param']] = param_value
        estimator = conf['estimator'](**conf['tuned_params'])
        print("Benchmarking %s" % estimator)
        estimator.fit(conf['data']['X_train'], conf['data']['y_train'])
        conf['postfit_hook'](estimator)
        complexity = conf['complexity_computer'](estimator)
        complexities.append(complexity)
        start_time = time.time()
        for _ in range(conf['n_samples']):
            y_pred = estimator.predict(conf['data']['X_test'])
        elapsed_time = (time.time() - start_time) / float(conf['n_samples'])
        prediction_times.append(elapsed_time)
        pred_score = conf['prediction_performance_computer'](
            conf['data']['y_test'], y_pred)
        prediction_powers.append(pred_score)
        print("Complexity: %d | %s: %.4f | Pred. Time: %fs\n" % (
            complexity, conf['prediction_performance_label'], pred_score,
            elapsed_time))
    return prediction_powers, prediction_times, complexities


def plot_influence(conf, mse_values, prediction_times, complexities):
    """
    Plot influence of model complexity on both accuracy and latency.
    """
    plt.figure(figsize=(12, 6))
    host = host_subplot(111, axes_class=Axes)
    plt.subplots_adjust(right=0.75)
    par1 = host.twinx()
    host.set_xlabel('Model Complexity (%s)' % conf['complexity_label'])
    y1_label = conf['prediction_performance_label']
    y2_label = "Time (s)"
    host.set_ylabel(y1_label)
    par1.set_ylabel(y2_label)
    p1, = host.plot(complexities, mse_values, 'b-', label="prediction error")
    p2, = par1.plot(complexities, prediction_times, 'r-',
                    label="latency")
    host.legend(loc='upper right')
    host.axis["left"].label.set_color(p1.get_color())
    par1.axis["right"].label.set_color(p2.get_color())
    plt.title('Influence of Model Complexity - %s' % conf['estimator'].__name__)
    plt.show()


def _count_nonzero_coefficients(estimator):
    a = estimator.coef_.toarray()
    return np.count_nonzero(a)

# #############################################################################
# Main code
#regression_data = generate_data('regression')
classification_data = generate_data('classification', sparse=True)
configurations = [
    {'estimator': SGDClassifier,
     'tuned_params': {'penalty': 'elasticnet', 'alpha': 0.001, 'loss':
                      'modified_huber', 'fit_intercept': True},
     'changing_param': 'l1_ratio',
     'changing_param_values': [0.25, 0.5, 0.75, 0.9],
     'complexity_label': 'non_zero coefficients',
     'complexity_computer': _count_nonzero_coefficients,
     'prediction_performance_computer': hamming_loss,
     'prediction_performance_label': 'Hamming Loss (Misclassification Ratio)',
     'postfit_hook': lambda x: x.sparsify(),
     'data': classification_data,
     'n_samples': 30},
   
]
for conf in configurations:
    prediction_performances, prediction_times, complexities = \
        benchmark_influence(conf)
    plot_influence(conf, prediction_performances, prediction_times,
                   complexities)

# feature importance

In [ ]:
#



In [ ]:
# Plot training data.

# Load data
X, y = dt1.f

title = "Training: Learning Curves (Naive Bayes)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
estimator = GaussianNB()
plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=40)

title = "Training: Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=2, test_size=0.02, random_state=0)
estimator = SVC(gamma=0.001)
plot_learning_curve(estimator, title, X, y, (0.7, 1.01), cv=cv, n_jobs=40)

plt.show()

In [ ]:
# Small data set
# Variance threshold (p=0.2).
X, y, subject = dt1.s
p = 0.2
sel = VarianceThreshold(threshold=(p*(1-p)))
X = sel.fit_transform(X)
print('V[X] =', '{0:.2f}'.format(p*(1-p)),
    ': num features =',len(X[0]))

title = "Training: Learning Curves (Naive Bayes) + VarianceThreshold"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
estimator = GaussianNB()
plot_learning_curve(estimator, title, X, y, ylim=(0.2, 1.01), cv=cv, n_jobs=40)

title = "Training: Learning Curves (SVM, RBF kernel, $\gamma=0.001$) + VarianceThreshold"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=1, test_size=0.02, random_state=0)
print('cv',cv)
estimator = SVC(gamma=0.001)
plot_learning_curve(estimator, title, X, y, (0.2, 1.01), cv=cv, n_jobs=40)

plt.show()

In [ ]:
# Small training set.
# Variance threshold (p=0.5).
X, y = dt1.s
p = 0.5
sel = VarianceThreshold(threshold=(p*(1-p)))
X = sel.fit_transform(X)
print('V[X] =', '{0:.2f}'.format(p*(1-p)),
    ': num features =',len(X[0]))

title = "Training: Learning Curves (Naive Bayes) + VarianceThreshold"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
estimator = GaussianNB()
plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=8)

title = "Training: Learning Curves (SVM, RBF kernel, $\gamma=0.001$) + VarianceThreshold"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=2, test_size=0.2, random_state=0)
estimator = SVC(gamma=0.001)
plot_learning_curve(estimator, title, X, y, (0.7, 1.01), cv=cv, n_jobs=8)

plt.show()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle

X, y = dt1.f

print('Initial data:',X.shape)
#(150, 4)
clf = ExtraTreesClassifier()
clf = clf.fit(X, y)
print('clf:',clf)

print('Feature importances:',clf.feature_importances_)
#array([ 0.04...,  0.05...,  0.4...,  0.4...])
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
print('New data:',X_new.shape)
#(150, 2)

clf = ExtraTreesClassifier()
clf = clf.fit(X_new, y)
print('New data feature importances:',clf.feature_importances_)




In [ ]:

# 1.13.5
from sklearn.svm import LinearSVC
import sklearn.pipeline
from sklearn.feature_selection import SelectFromModel
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
  ('classification', RandomForestClassifier())
])
clf.fit(X, y)
